In [38]:
import json
sys.path.append('../')

from utils import (
    load_json,jsonl_dir_to_df,
    load_simple_json,generate_evidence_to_wiki_pages_mapping
)


In [116]:
with open("pred_enough_n.txt","r")as f:
    pred_enough = f.readlines()
enough_dic ={}
for i in pred_enough:
    buf = i.replace("\n","").split(" ")
    enough_dic[int(buf[0])]=buf[1]
len(enough_dic.keys())

9038

In [145]:
train_data = load_json("../preprocess/pre_all_0522.jsonl")
len(train_data)

11349

In [156]:
test_data = load_json("../preprocess/public_test.jsonl")+load_json("../preprocess/private_test_data.jsonl")
test_map = {i["id"]:idx for idx,i in enumerate(test_data)}
len(test_data)

9038

In [398]:
pred_data = load_json("./all_test_clu_page_sent_claim_nclu_ens5_probs.jsonl")
pred_map = {i["id"]:idx for idx,i in enumerate(pred_data)}
len(pred_data)

8597

In [324]:
len([i for i in pred_data if i["enough"]<0.1])#not enough predict

909

In [104]:
def find_identical(data,test):
    re_lst=[]
    for i in test:
        for j in data:
            if i["claim"].replace(" ","") == j["claim"].replace(" ",""):
                re_lst.append({"id":i["id"],"label":j["label"]})
    return re_lst

In [146]:
script = find_identical(train_data,test_data)
len(script)

136

In [402]:
for i in script:
    if i["id"] in pred_map:
        if pred_data[pred_map[i["id"]]]["predicted_label"]!=i["label"]:
            print(i)
            pred_data[pred_map[i["id"]]]["predicted_label"]=i["label"]

{'id': 2045, 'label': 'supports'}
{'id': 300, 'label': 'refutes'}
{'id': 208, 'label': 'NOT ENOUGH INFO'}
{'id': 207, 'label': 'NOT ENOUGH INFO'}
{'id': 6028, 'label': 'refutes'}
{'id': 689, 'label': 'NOT ENOUGH INFO'}
{'id': 4082, 'label': 'NOT ENOUGH INFO'}
{'id': 223, 'label': 'NOT ENOUGH INFO'}
{'id': 10544, 'label': 'supports'}


In [399]:
for i in enough_dic:
    if i in pred_map.keys():
        if enough_dic[i]=="NOT":
            pred_data[pred_map[i]]['cluster_enough']=0
        else:
            pred_data[pred_map[i]]['cluster_enough']=1
    else:
        0
        #print(i,"NOT IN Pred")

In [236]:
pred_data[0]

{'id': 21498,
 'claim': '雞形目的鳥腿腳強健，擅長在地面奔跑，其中有珍稀物種，體態雄健優美、顏色鮮豔；也有經濟物種，與人類關係密切。',
 'predicted_pages_1': ['雞形目'],
 'predicted_evidence': [['雞形目', 2], ['雞形目', 3], ['雞形目', 0], ['雞形目', 1]],
 'enough': 0.9632043480873108,
 'predict_label': 0.009029662888497113,
 'cluster_enough': 1}

In [400]:
count=0
for i in pred_data:#if enough not sure use cluster
    if abs(i["enough"]-0.5)<0.1:
        count+=1
        i["enough"]=i['cluster_enough']
    else:
        i["enough"]=1 if i["enough"]>0.5 else 0
print(count)

443


In [401]:
c_lst=[]
for i in pred_data:#mapping to labels
    if i["enough"]:
        if i['predict_label']>0.5:
            i["predicted_label"]="refutes"
        else:
            i["predicted_label"]="supports"
    else:
        i["predicted_label"]="NOT ENOUGH INFO"
    c_lst.append(i["predicted_label"])
print("total: ",len(c_lst))
print("ratio: ",c_lst.count("supports"),c_lst.count("refutes"),c_lst.count("NOT ENOUGH INFO"))


total:  8597
ratio:  4042 2531 2024


all ratio:  3298 2323 2976 c8597
01 ratio:  4042 2531 2024 c443
02 ratio:  4063 2593 1941 c851
03 ratio:  3982 2533 2082 c1536
04 ratio:  4009 2599 1989 c2334
045 ratio:  3956 2558 2083 c2758

kfold 0.5 1904
kfold 0.4 1904

add not found data

In [240]:
ori_data = load_json("../preprocess/all_test.jsonl")
len(ori_data)

9038

In [403]:
mapping_idx = {d["id"]:idx for idx,d in enumerate(pred_data)}
re_lst = []
for d in ori_data:
    if d["id"] in mapping_idx.keys():
        data = pred_data[mapping_idx[d["id"]]]
        re_lst.append({"id":data["id"],"predicted_label":data["predicted_label"],"predicted_evidence":data["predicted_evidence"]})
    else:
        re_lst.append({"id":d["id"],"predicted_label":"NOT ENOUGH INFO","predicted_evidence":[]})

In [404]:
len(re_lst)

9038

In [406]:
with open("../submissions/all_test_clu_page_sent_claim_cluE01_ens5.jsonl","w",encoding="utf8",) as f:
    for i, d in enumerate(re_lst):
        f.write(json.dumps(d, ensure_ascii=False) + "\n")

In [405]:
print([ i for i in re_lst if len(i.keys())>3])
len([ i for i in re_lst if i["predicted_label"]=="NOT ENOUGH INFO"])

[]


2468

In [155]:
for i in pred_map:
    if i not in test_map.keys():
        print(i)